In [ ]:
%load_ext watermark
%watermark -v -n -m -p numpy,scipy,sklearn,pandas

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
os.chdir(PROJ_ROOT)
import configparser

In [3]:
import ast
import pandas as pd

In [5]:
sys.path.append('.')
from src.models.rf import RandomForestModel
from src.data import read_raw_data
from src.features.build_features import process_data, get_pol_value_series, lag_search_features, get_feature_array
import numpy as np
import ast

In [4]:
# import sys
# sys.path.append(os.path.join(PROJ_ROOT, 'src'))
# %aimport data.extract_search_trend
# from data.extract_search_trend import extract_search_trend as extract

In [6]:
config_path = 'config/parameters.ini'
train_data_path = 'data/processed/train.csv'
output_filepath = 'models/rf_model.pkl'
# train_data_path 
# test_data_path

In [7]:
    pars = configparser.ConfigParser()
    pars.read(config_path)

    seq_length = int(pars['train_model']['seq_length'])
    search_lag = int(pars['train_model']['search_lag'])

In [10]:
    # parameters for rf model
    n_estimators = ast.literal_eval(pars['train_model']['n_estimators'])
    max_depth = ast.literal_eval(pars['train_model']['max_depth'])

    train_data = read_raw_data(train_data_path)
    y_train, pol_val, trend_fea, phys_fea = process_data(train_data)
    processed_pol = get_pol_value_series(pol_val, seq_length)
    processed_trend = lag_search_features(trend_fea, search_lag)
    process_phys = np.array(phys_fea)

In [16]:
phys_fea

,DATE,TEMP 1st Max Value,TEMP Mean,RH Mean,quad_temp,cubic_temp,dew_point,quad_dew,cubic_dew,heat
0,2007-03-01,-1.393080,-1.133650,1.267300,-1.394197,-1.370629,-1.124761,-1.239207,-1.053899,0.433176
1,2007-03-02,-1.663528,-1.632591,-2.264214,-1.618267,-1.551679,-1.633357,-1.381732,-1.086330,1.438255
2,2007-03-03,-1.753677,-1.595269,-3.243574,-1.690604,-1.608278,-1.595047,-1.376866,-1.085879,1.360098
3,2007-03-04,-2.835466,-2.750298,-2.048949,-2.466909,-2.153626,-2.807439,-1.067141,-1.189268,3.549505
4,2007-03-05,-1.753677,-2.255286,-2.452329,-1.690604,-1.608278,-2.280253,-1.319500,-1.097648,2.713930
...,...,...,...,...,...,...,...,...,...,...
730,2009-10-27,-2.024124,-1.308476,1.897581,-1.900559,-1.767240,-1.302140,-1.308054,-1.073476,0.772111
731,2009-10-28,-0.942334,-0.825249,1.344873,-0.997224,-1.029575,-0.813757,-1.069013,-0.985393,-0.104558
732,2009-10-29,-0.581738,-0.567921,0.807679,-0.658473,-0.719202,-0.555896,-0.880113,-0.884318,-0.472074
733,2009-10-30,-1.393080,-0.752568,1.938307,-1.394197,-1.370629,-0.740784,-1.019951,-0.961395,-0.216898


In [ ]:




    x_train, embedding_dim = get_feature_array(processed_pol, processed_trend, process_phys, seq_length)

    model = RandomForestModel(n_estimators, max_depth)
    model.fit(x_train, x_train, y_train, y_train)
    model.save(output_filepath)

In [14]:
    pars = configparser.ConfigParser()
    pars.read(config_path)
    train_years = ast.literal_eval(pars['global']['train_year'])
    test_years = ast.literal_eval(pars['global']['test_year'])

    merged_data = read_raw_data(merged_file_path)
    merged_data.index = pd.to_datetime(merged_data.DATE)

In [9]:
train_data = select_years(merged_data, train_years)

In [26]:
import datetime as dt

In [27]:
# select single year data
def select_single_year(input_df, single_year):
    """

    :param input_df: pd.DataFrame
    :param single_year: int
    :return: list(int)
    """
    start = input_df.index.searchsorted(dt.date(single_year, 1, 1))
    end = input_df.index.searchsorted(dt.date(single_year, 12, 31))
    single_year_index = [k for k in range(start, end + 1)]
    return single_year_index

In [28]:
test_data = select_single_year(merged_data, 2011)

In [36]:
end = merged_data.index.searchsorted(dt.date(2007, 1, 1))

In [39]:
dt.date(2007, 3, 1) in merged_data.index

True

In [34]:
merged_data

,Ozone_median_class,headache,chest pain,cough,smoke,ozone,DATE,TEMP 1st Max Value,TEMP Mean,RH Mean,quad_temp,cubic_temp,dew_point,quad_dew,cubic_dew,heat
DATE,,,,,,,,,,,,,,,,
2007-03-01,0,35.000000,31.000000,80.000000,47.000000,51.000000,2007-03-01,65.0,57.041667,85.229166,4225.0,274625.0,5.542095,30.714819,170.224447,86.452797
2007-03-02,0,38.000000,33.000000,21.000000,20.000000,0.000000,2007-03-02,62.0,51.750000,47.291667,3844.0,238328.0,1.423587,2.026599,2.885039,95.760483
2007-03-03,0,58.000000,0.000000,32.000000,23.000000,42.000000,2007-03-03,61.0,52.145834,36.770834,3721.0,226981.0,1.733812,3.006104,5.212020,95.036697
2007-03-04,0,0.000000,0.000000,65.000000,23.000000,43.000000,2007-03-04,49.0,39.895834,49.604167,2401.0,117649.0,-8.083874,65.349023,-528.273287,115.312041
2007-03-05,0,37.000000,0.000000,20.000000,10.000000,0.000000,2007-03-05,61.0,45.145834,45.270834,3721.0,226981.0,-3.814840,14.553005,-55.517386,107.574065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-10-27,0,44.872222,41.969084,37.518405,56.267281,34.874468,2011-10-27,76.0,61.625000,74.125000,5776.0,438976.0,9.066140,82.194887,745.190314,79.737278
2011-10-28,0,45.966667,64.265161,37.518405,57.290323,34.874468,2011-10-28,71.0,58.062500,85.416666,5041.0,357911.0,6.330119,40.070410,253.650477,84.811547
2011-10-29,0,86.461111,24.919144,68.088957,63.428571,50.897872,2011-10-29,59.0,48.062500,74.270833,3481.0,205379.0,-1.485754,2.207466,-3.279751,102.491902


In [29]:
test_data

[980,
 981,
 982,
 983,
 984,
 985,
 986,
 987,
 988,
 989,
 990,
 991,
 992,
 993,
 994,
 995,
 996,
 997,
 998,
 999,
 1000,
 1001,
 1002,
 1003,
 1004,
 1005,
 1006,
 1007,
 1008,
 1009,
 1010,
 1011,
 1012,
 1013,
 1014,
 1015,
 1016,
 1017,
 1018,
 1019,
 1020,
 1021,
 1022,
 1023,
 1024,
 1025,
 1026,
 1027,
 1028,
 1029,
 1030,
 1031,
 1032,
 1033,
 1034,
 1035,
 1036,
 1037,
 1038,
 1039,
 1040,
 1041,
 1042,
 1043,
 1044,
 1045,
 1046,
 1047,
 1048,
 1049,
 1050,
 1051,
 1052,
 1053,
 1054,
 1055,
 1056,
 1057,
 1058,
 1059,
 1060,
 1061,
 1062,
 1063,
 1064,
 1065,
 1066,
 1067,
 1068,
 1069,
 1070,
 1071,
 1072,
 1073,
 1074,
 1075,
 1076,
 1077,
 1078,
 1079,
 1080,
 1081,
 1082,
 1083,
 1084,
 1085,
 1086,
 1087,
 1088,
 1089,
 1090,
 1091,
 1092,
 1093,
 1094,
 1095,
 1096,
 1097,
 1098,
 1099,
 1100,
 1101,
 1102,
 1103,
 1104,
 1105,
 1106,
 1107,
 1108,
 1109,
 1110,
 1111,
 1112,
 1113,
 1114,
 1115,
 1116,
 1117,
 1118,
 1119,
 1120,
 1121,
 1122,
 1123,
 1124,
 1125

In [23]:
len(merged_data)

1225

In [15]:
 test_data = select_years(merged_data, test_years)

IndexError: positional indexers are out-of-bounds

In [12]:
test_years

[2010, 2011]

In [13]:
merged_data

,Ozone_median_class,headache,chest pain,cough,smoke,ozone,DATE,TEMP 1st Max Value,TEMP Mean,RH Mean,quad_temp,cubic_temp,dew_point,quad_dew,cubic_dew,heat
DATE,,,,,,,,,,,,,,,,
2007-03-01,0,35.000000,31.000000,80.000000,47.000000,51.000000,2007-03-01,65.0,57.041667,85.229166,4225.0,274625.0,5.542095,30.714819,170.224447,86.452797
2007-03-02,0,38.000000,33.000000,21.000000,20.000000,0.000000,2007-03-02,62.0,51.750000,47.291667,3844.0,238328.0,1.423587,2.026599,2.885039,95.760483
2007-03-03,0,58.000000,0.000000,32.000000,23.000000,42.000000,2007-03-03,61.0,52.145834,36.770834,3721.0,226981.0,1.733812,3.006104,5.212020,95.036697
2007-03-04,0,0.000000,0.000000,65.000000,23.000000,43.000000,2007-03-04,49.0,39.895834,49.604167,2401.0,117649.0,-8.083874,65.349023,-528.273287,115.312041
2007-03-05,0,37.000000,0.000000,20.000000,10.000000,0.000000,2007-03-05,61.0,45.145834,45.270834,3721.0,226981.0,-3.814840,14.553005,-55.517386,107.574065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-10-27,0,44.872222,41.969084,37.518405,56.267281,34.874468,2011-10-27,76.0,61.625000,74.125000,5776.0,438976.0,9.066140,82.194887,745.190314,79.737278
2011-10-28,0,45.966667,64.265161,37.518405,57.290323,34.874468,2011-10-28,71.0,58.062500,85.416666,5041.0,357911.0,6.330119,40.070410,253.650477,84.811547
2011-10-29,0,86.461111,24.919144,68.088957,63.428571,50.897872,2011-10-29,59.0,48.062500,74.270833,3481.0,205379.0,-1.485754,2.207466,-3.279751,102.491902


In [39]:
years, trend_data = extract_search_trend(config_path, output_filepath)

In [40]:
years

[2007, 2008, 2009, 2010, 2011]

In [41]:
trend_data

,Unnamed: 0,advair,air pollutant,air pollution,albuterol,allergies,allergy,apnea,arrhythmia,ash,...,tinnitus,traffic,vapors,ventolin,vomiting,wheeze,wheezes,wheezing,wheezy,wildfires
0,2007-01-01,0.00,0.0,0.0000,0.000000,58.000000,55.000000,0.000000,0.000000,20.000000,...,0.000000,6.000000,0.000000,0.00000,57.000000,0.000000,NaN,0.000000,0.000000,0.000000
1,2007-01-02,37.00,0.0,0.0000,0.000000,52.000000,16.000000,36.000000,0.000000,9.000000,...,0.000000,14.000000,0.000000,0.00000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000
2,2007-01-03,0.00,0.0,0.0000,0.000000,0.000000,0.000000,35.000000,0.000000,9.000000,...,0.000000,20.000000,0.000000,0.00000,32.000000,0.000000,NaN,0.000000,0.000000,0.000000
3,2007-01-04,0.00,0.0,0.0000,33.000000,50.000000,47.000000,35.000000,0.000000,6.000000,...,0.000000,26.000000,0.000000,0.00000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000
4,2007-01-05,0.00,0.0,0.0000,0.000000,0.000000,63.000000,0.000000,0.000000,12.000000,...,33.000000,38.000000,0.000000,0.00000,0.000000,0.000000,NaN,62.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4409,2019-01-27,0.00,0.0,25.2000,31.589365,221.176471,59.640662,19.284202,0.000000,509.134078,...,89.944050,37.756789,0.000000,0.00000,43.891269,0.000000,0.0,22.378208,0.000000,0.000000
4410,2019-01-28,0.00,0.0,40.1625,24.938972,69.117647,78.100867,54.449511,22.572657,499.877095,...,109.431927,122.709563,54.899637,25.01279,34.913509,35.783784,0.0,0.000000,0.000000,0.679534
4411,2019-01-29,0.00,0.0,28.3500,11.638187,129.941176,76.680851,88.480456,21.632130,620.217877,...,58.463632,90.616293,25.769217,0.00000,20.948105,0.000000,0.0,35.805134,0.000000,0.000000
4412,2019-01-30,0.00,85.0,48.0375,29.926767,91.235294,69.580772,104.361564,43.264260,518.391061,...,116.927265,98.167651,26.889618,47.43805,37.906096,34.079794,0.0,17.902567,0.000000,0.271814


In [11]:
import pandas as pd

In [13]:
a = pd.DataFrame([1,2,3,4])

In [15]:
a.iloc[[0,2], :]

,0
0,1
2,3


In [18]:
date_rng = pd.date_range('20060101','20060105',freq='D')


In [26]:
int(pd.Series(date_rng.format())[0].split('-')[1])

1

In [20]:
date_rng

DatetimeIndex(['2006-01-01', '2006-01-02', '2006-01-03', '2006-01-04',
               '2006-01-05'],
              dtype='datetime64[ns]', freq='D')

In [17]:
RECORD_COLUMNS = ['accuracy', 'F1 score', 'true positives', 'false positives', 'true negatives',
                  'false negatives', 'AUC']

In [30]:
np.array(a).reshape(1,-1)

array([[  0.76122449,   0.70229008, 138.        ,  88.        ,
        235.        ,  29.        ,   0.85734414]])

In [32]:
record_pd = pd.DataFrame(np.array(a).reshape(1,-1), columns=RECORD_COLUMNS)

In [33]:
record_pd

,accuracy,F1 score,true positives,false positives,true negatives,false negatives,AUC
0,0.761224,0.70229,138.0,88.0,235.0,29.0,0.857344


In [19]:
a = (0.7612244897959184, 0.7022900763358779, 138, 88, 235, 29, 0.857344135258894)

In [26]:
record_pd.iloc[0, :] 

IndexError: single positional indexer is out-of-bounds

In [34]:
# os.makedirs('')

In [36]:
open('./notebooks/a.txt', 'w').close()